In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 KB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 1

In [3]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.9 MB/s eta 0:00:00


In [4]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import interpret 
from interpret import show
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.data import Marginal


In [5]:
dpath = 'drive/MyDrive/Seculayer/OSC/data/'

train = pd.read_csv(dpath+'train.csv')
test = pd.read_csv(dpath+'test.csv')

In [6]:
def make_missing_column(df,columns):
    df['{}_missing'.format(columns)] = np.where(df[columns].isna()==True, 1, 0)
    return df 
    
def columns_with_missing_values(df):
    missing_cols = df.columns[df.isna().any()].tolist()
    return missing_cols

In [7]:
missing_cols = columns_with_missing_values(train)

for item in missing_cols:
    train = make_missing_column(train,item)

train.dropna(axis=1,inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      14095 non-null  object 
 1   COMPONENT_ARBITRARY     14095 non-null  object 
 2   ANONYMOUS_1             14095 non-null  int64  
 3   YEAR                    14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY     14095 non-null  int64  
 5   ANONYMOUS_2             14095 non-null  int64  
 6   AG                      14095 non-null  int64  
 7   AL                      14095 non-null  int64  
 8   B                       14095 non-null  int64  
 9   BA                      14095 non-null  int64  
 10  BE                      14095 non-null  int64  
 11  CA                      14095 non-null  int64  
 12  CO                      14095 non-null  int64  
 13  CR                      14095 non-null  int64  
 14  CU                      14095 non-null

In [8]:
X = train.drop(['ID'],axis=1)

le1 = LabelEncoder()
le2 = LabelEncoder()
X['COMPONENT_ARBITRARY_cat'] = le1.fit_transform(X['COMPONENT_ARBITRARY'])
X['YEAR_cat'] = le2.fit_transform(X['YEAR'])
X.drop(['YEAR','COMPONENT_ARBITRARY'],axis=1,inplace=True)
X['COMPONENT_ARBITRARY_cat'] = X['COMPONENT_ARBITRARY_cat'].astype('category')
X['YEAR_cat'] = X['YEAR_cat'].astype('category')

y = X['Y_LABEL']
X.drop('Y_LABEL',axis=1,inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3,stratify=y)

In [9]:
X

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,...,U75_missing,U50_missing,U25_missing,U20_missing,U14_missing,U6_missing,U4_missing,V100_missing,COMPONENT_ARBITRARY_cat,YEAR_cat
0,1486,7,200,0,3,93,0,0,3059,0,...,1,1,1,1,1,1,1,1,2,4
1,1350,51,375,0,2,19,0,0,2978,0,...,0,0,0,0,0,0,0,1,1,14
2,2415,2,200,0,110,1,1,0,17,0,...,0,0,0,0,0,0,0,1,1,8
3,7389,2,200,0,8,3,0,0,1960,0,...,1,1,1,1,1,1,1,1,2,3
4,3954,4,200,0,1,157,0,0,71,0,...,1,1,1,1,1,1,1,1,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,1616,8,200,0,2,201,1,0,6,0,...,1,1,1,1,1,1,1,1,2,7
14091,2784,2,200,0,3,85,0,0,2945,0,...,1,1,1,1,1,1,1,0,0,6
14092,1788,9,550,0,6,0,1,0,13,0,...,1,1,1,1,1,1,1,1,2,1
14093,2498,19,550,0,2,4,0,0,2244,0,...,0,0,0,0,0,0,0,1,1,2


In [ ]:
from interpret import show

marginal = Marginal().explain_data(X_train.iloc[:,:-2], y_train.iloc[:,:-2], name = 'Train Data')
show(marginal)

In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "max_leaves" : trial.suggest_int("max_leaves", 2, 16),
        "binning": trial.suggest_categorical("binning",["uniform","quantile","rounded_quantile"]),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds",50,500)
  }
    
    model = ExplainableBoostingClassifier(**params_cat)
    model.fit(X_train, y_train)

    cat_pred = model.predict(X_val)
    F1 = f1_score(y_val, cat_pred)
    
    return F1

In [ ]:
# Optuna

sampler = TPESampler(seed = 2023)
study = optuna.create_study(
    study_name = "ebm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2023-02-08 05:28:35,099] A new study created in memory with name: ebm_parameter_opt
<ipython-input-25-d8cfe1e2ad38>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-08 05:29:16,704] Trial 0 finished with value: 0.6309963099630995 and parameters: {'learning_rate': 0.009246234676674875, 'max_depth': 15, 'binning': 'uniform', 'early_stopping_rounds': 261}. Best is trial 0 with value: 0.6309963099630995.
<ipython-input-25-d8cfe1e2ad38>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
[I 2023-02-

KeyboardInterrupt: ignored

In [ ]:
ebm_param = {'learning_rate': 0.007217397831855597, 'max_leaves': 2, 'binning': 'quantile', 'early_stopping_rounds': 451}

ebm = ExplainableBoostingClassifier(**ebm_param,random_state=2023)
ebm.fit(X_train, y_train)


# Global Explanations: What the model learned overall
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

In [ ]:
f1_score(y_val,ebm.predict(X_val))

0.667870036101083

In [ ]:
f1_score(y_val,ebm.predict(X_val))

0.6923076923076924

In [ ]:
test.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

AL, CA가 중요한 Feature임을 확인

In [ ]:
importances = pd.DataFrame()
importances['feature'] = ebm_global.data()['names']
importances['score'] = ebm_global.data()['scores']
importances.sort_values('score',ascending=False).head(11)

,feature,score
4,AL,0.632965
0,ANONYMOUS_1,0.500053
51,YEAR_cat,0.401787
8,CA,0.395584
57,AL & CA,0.264197
12,FE,0.258332
6,BA,0.255072
59,CA & YEAR_cat,0.199063
20,P,0.194441
54,ANONYMOUS_1 & YEAR_cat,0.180946


In [ ]:
importances.sort_values('score',ascending=False)

,feature,score
4,AL,0.632965
0,ANONYMOUS_1,0.500053
51,YEAR_cat,0.401787
8,CA,0.395584
57,AL & CA,0.264197
...,...,...
36,FSO4_missing,0.002930
37,FTBN_missing,0.002907
38,FUEL_missing,0.002884
40,SOOTPERCENTAGE_missing,0.002784
